**//IMPORTS**

In [ ]:
import os, gzip, requests, time
import pandas as pd
import pathlib, sys

from datetime import datetime, timezone
from dotenv import load_dotenv         
from pathlib import Path
from opensubtitlescom import OpenSubtitles
from xmlrpc.client import ServerProxy, Error as XMLRPCError

project_root = pathlib.Path().resolve().parents[0] 
print(f"Project root: {project_root}")
sys.path.insert(0, str(project_root / "src"))
from db import connect     
from extract.access_open_subtitles import download_srt
from transform.align_subtitles import eliminate_new_lines, auto_sync_subs, align_subtitles_optimal_hungarian
from load.load_subtitles import load_subtitles
from catalog import collect_movies

Project root: /home/rofarate/Thesis


**//CONFIGS**

In [24]:
load_dotenv()

API_BASE = "https://api.opensubtitles.com/api/v1"
API_KEY = os.getenv("OPENSUBTITLES_API_KEY")
USERNAME = os.getenv("OPENSUBTITLES_USER", "")
PASSWORD = os.getenv("OPENSUBTITLES_PASS", "")
USER_AGENT = "MySubtitleApp/1.0"

# REST headers
# REST_HEADERS = {
#     "Api-Key":     API_KEY,
#     "User-Agent":  USER_AGENT,
#     "Content-Type":"application/json"
# }

# HEAD = {
#     "Api-Key": os.getenv("OPENSUBTITLES_API_KEY"),
#     "User-Agent": "TeseCollector/0.1",
#     "Accept": "application/json",
# }

BASE_HEADERS = {
    "Api-Key":     API_KEY,
    "User-Agent":  "MySubtitleApp/1.0",          # <- descriptive!
    "Accept":      "application/json",           # <- important
    "Content-Type":"application/json",
}
AUTH_HEADERS = dict(BASE_HEADERS)       # will gain 'Authorization' below

# XML-RPC client (fallback path)
ost = OpenSubtitles(user_agent=USER_AGENT, api_key=API_KEY)
# optional login for higher quotas
if USERNAME and PASSWORD:
    try:
        ost.login(USERNAME, PASSWORD)
    except Exception:
        pass

# YEARS AND LANGUAGES

YEARS = range(2023, 2024)          
LANGS = {"pt-br", "pt-pt"}             
by_lang = {lang: {} for lang in LANGS}

TIMEOUT = 15


In [25]:
from pathlib import Path

# create the directories we’ll use
p = Path("../data/duckdb")
p.mkdir(parents=True, exist_ok=True)

# lightweight runtime config (will move to etl/config.py later)
from pathlib import Path

DUCKDB_PATH = Path("../data/duckdb/subs.duckdb")
PROCESSED_DIR = Path("../data/processed")        # <-- where anything derived lands
TIME_WINDOW = 60  

# OpenSubtitles token (if you’ve sourced .env already)
import os
OS_API_KEY = os.getenv("OPENSUBTITLES_API_KEY") 

In [26]:
def td_to_srt(ts) -> str:
    """
    Convert a pandas Timedelta *or* an integer millisecond offset
    to the SRT time-stamp string  HH:MM:SS,mmm
    """
    if isinstance(ts, (int, float)):             # already ms
        total_ms = int(ts)
    else:                                        # Timedelta  → ms
        total_ms = int(ts.total_seconds() * 1000)

    hours,   rem = divmod(total_ms, 3_600_000)
    minutes, rem = divmod(rem,        60_000)
    seconds, ms  = divmod(rem,         1_000)
    return f"{hours:02}:{minutes:02}:{seconds:02},{ms:03}"


**//MAIN CODE**

In [27]:
con = connect()                     # ← runs the DDL, makes subs.duckdb
print(con.execute("PRAGMA show_tables").fetchall())
con.close()

[('movies',), ('subtitle_pairs',)]


In [28]:
total = collect_movies(range(2023, 2024))      # 2023 only
print("movies in DB:", total)


movies in DB: 672


In [36]:
con = connect()    
movies = con.execute("SELECT imdb_id, title FROM movies").fetchall()
len(movies)

672

In [37]:
from transform.align_subtitles import merge_subtitle_fragments

movies = movies[:10]
print(movies)

for imdb_id, title in movies:
    try:
        movie_id = con.execute(
            "SELECT movie_id FROM movies WHERE imdb_id = ?", 
            [imdb_id]
        ).fetchone()[0]

        # 2) skip if we've already stored pairs for this movie
        already = con.execute(
            "SELECT COUNT(*) FROM subtitle_pairs WHERE movie_id = ?", 
            [movie_id]
        ).fetchone()[0]
        if already:
            print(f"→ Skipping {imdb_id} ({title}), already in DB")
            continue
        # ---- download or reuse cache ------------------------------------
        pt_path = download_srt(imdb_id, "pt-PT", title=title)
        br_path = download_srt(imdb_id, "pt-BR", title=title)

        # ---- load & quick clean -----------------------------------------
        subs_pt = load_subtitles(pt_path)
        subs_br = load_subtitles(br_path)
        eliminate_new_lines(subs_pt)
        eliminate_new_lines(subs_br)

        # ---- sentence-level merge (optional but helps alignment) --------
        subs_pt = merge_subtitle_fragments(subs_pt, gap_threshold=pd.Timedelta(seconds=0.2))
        subs_br = merge_subtitle_fragments(subs_br, gap_threshold=pd.Timedelta(seconds=0.2))

        # ---- auto sync --------------------------------------------------
        shifted_pt, final_offset = auto_sync_subs(subs_pt, subs_br)

        # ---- fine alignment --------------------------------------------
        pairs = align_subtitles_optimal_hungarian(shifted_pt, subs_br)

        rows = []
        pair_no = 1
        for pt, br, score in pairs:
            if br is None:
                continue                   # skip unmatched lines

            rows.append({
                "imdb_id": imdb_id,
                "pair_no":  pair_no,
                #"pair_no":  pair_no,
                # store the raw millis             (handy for SQL filtering)
                "start_pt_ms": int(pt["start"].total_seconds() * 1000),
                "end_pt_ms":   int(pt["end"].total_seconds()   * 1000),
                "start_br_ms": int(br["start"].total_seconds() * 1000),
                "end_br_ms":   int(br["end"].total_seconds()   * 1000),
                # and the pretty SRT strings       (for export / debugging)
                # "start_pt_ms": td_to_srt(pt["start"]),
                # "end_pt_ms":   td_to_srt(pt["end"]),
                # "start_br_ms": td_to_srt(br["start"]),
                # "end_br_ms":   td_to_srt(br["end"]),
                "text_pt": pt["text"],
                "text_br": br["text"],
                "score":   float(score),
            })
            pair_no += 1

        pairs_df = pd.DataFrame(rows)
        #con.execute("DELETE FROM subtitle_pairs WHERE imdb_id = ?", [imdb_id])
        con.register("pairs_df", pairs_df)

        con.execute("""
            INSERT INTO subtitle_pairs (
            movie_id,
            pair_no,
            start_pt_ms,
            end_pt_ms,
            text_pt,
            start_br_ms,
            end_br_ms,
            text_br,
            score
            )
            SELECT
            m.movie_id,
            p.pair_no,
            p.start_pt_ms,
            p.end_pt_ms,
            p.text_pt,
            p.start_br_ms,
            p.end_br_ms,
            p.text_br,
            p.score
            FROM pairs_df AS p
            JOIN movies   AS m
            ON p.imdb_id = m.imdb_id
            """)
        
        print(f"✓ Movie {imdb_id} ({title}) in the database")

    except RuntimeError as e:        # e.g. no subtitles, 5xx after retries
        print("✗", imdb_id, title, "→", e)
    except Exception as e:
        print("⚠️  unexpected failure on", imdb_id, ":", e)

[('14230458', 'Poor Things'), ('15764854', 'Fireworks'), ('26449465', 'Peak Season'), ('19883634', 'The Old Oak'), ('27805677', 'Fairlane'), ('20465746', "L'ultima notte di Amore"), ('26671415', 'Seven Veils'), ('22687790', 'A Haunting in Venice'), ('5478456', 'We Are Zombies'), ('18072882', 'Northern Comfort')]
→ Skipping 14230458 (Poor Things), already in DB
→ Skipping 15764854 (Fireworks), already in DB
→ Skipping 26449465 (Peak Season), already in DB
Offset +0.000s (max 1.161)  Coverage 0.101  Mean 0.979
Offset +0.000s (max 1.161)  Coverage 0.101  Mean 0.979
Offset +0.000s (max 1.161)  Coverage 0.101  Mean 0.979
plateau → stop
✓ Movie 19883634 (The Old Oak) in the database
Download refused: {'requests': 6, 'remaining': -1, 'message': 'You have downloaded your allowed 5 subtitles for 24h.Your quota will be renewed in 08 hours and 46 minutes (2025-07-08 23:59:58 UTC) ts=1751987592 ', 'reset_time': '08 hours and 46 minutes', 'reset_time_utc': '2025-07-08T23:59:58.000Z', 'uk': 'app_125

In [40]:
df_subtitle_pairs = con.execute("SELECT * FROM subtitle_pairs").df()
low_scores_in_db = df_subtitle_pairs[df_subtitle_pairs.score < 0.5]
high_scores_in_db = df_subtitle_pairs[df_subtitle_pairs.score >= 0.7]


In [41]:
good_row_count = len(high_scores_in_db)
print(f"Subtitle pairs in DB: {good_row_count}")
high_scores_in_db

Subtitle pairs in DB: 2578


,pair_id,movie_id,pair_no,start_pt_ms,end_pt_ms,text_pt,start_br_ms,end_br_ms,text_br,score
0,1,1,1,980,5938,CONTÉM REPRESENTAÇÕES DE PRODUTOS DE TABACO,1000,6000,HÁ CENAS DE USO DE TABACO,0.712900
1,2,1,2,111772,118272,POBRES CRIATURAS,111800,118100,POBRES CRIATURAS,0.999860
2,3,1,3,286522,288480,Adeus!,286542,288500,Adeus.,0.880900
5,6,1,6,301147,305605,Quem gostaria de reconstruir os órgãos?,301167,305625,"Bem, quem gostaria de recolocar os órgãos?",0.852900
6,7,1,7,305772,310772,"E quem consegue distinguir humano de animal, i...",305792,310792,"E quem pode distinguir o humano do animal, se ...",0.873900
...,...,...,...,...,...,...,...,...,...,...
5250,5251,4,1347,6464442,6467567,"Obrigada por ter vindo. Obrigada, muito obrigada.",6464333,6467458,Obrigada por terem vindo. Obrigada. Muito obri...,0.957455
5251,5252,4,1348,6468748,6471849,"Muito obrigada. Obrigada, a sério. Significa m...",6469599,6471625,Obrigada mesmo. Significa muito para a gente. ...,0.764745
5252,5253,4,1349,6528250,6529250,Obrigada.,6528250,6529250,Obrigada.,1.000000
5253,5254,4,1350,6567042,6568333,"<i>Shukran</i>, Sr. Ballantyne.",6567042,6568333,"Shukran, Sr. Ballantyne.",0.909000


In [43]:
bad_row_count = len(low_scores_in_db)
print(f"Low-score pairs in DB: {bad_row_count}")
low_scores_in_db

Low-score pairs in DB: 557


,pair_id,movie_id,pair_no,start_pt_ms,end_pt_ms,text_pt,start_br_ms,end_br_ms,text_br,score
24,25,1,25,373563,374397,Ia gostar bastante.,373583,374417,Adoraria isso.,0.467900
192,193,1,193,1352272,1353355,E para sempre.,1363667,1367458,ou prisões mal equipadas e poucos funcionários?,0.355025
293,294,1,294,2032355,2034063,É bastante bom!,2032375,2034083,Ficou bem e ridículo.,0.495900
299,300,1,300,2052230,2053730,Compreendo porquê.,2048458,2049708,Entendi.,0.498140
401,402,1,402,2526522,2527688,Uma pândega!,2526542,2527708,Delícia.,0.369900
...,...,...,...,...,...,...,...,...,...,...
5163,5164,4,1260,5958497,5960201,Tentámos ajudar famílias,5990562,5993042,Eu vi no rosto das crianças.,0.405675
5191,5192,4,1288,6053725,6055905,"-Pá... -TJ, pá...",6053250,6057900,"- Ora, TJ, volta aqui. - Ei, deixa ele ir.",0.486625
5192,5193,4,1289,6056474,6058865,"-Então, TJ? -Joe. Joe, deixa-o.",6016770,6018792,Não passam de capachos.,0.332480
5210,5211,4,1307,6237854,6239562,Não duvido.,6240333,6242125,Com toda certeza.,0.490605
